Filename is pcc_resnet50_X.h5 (where X is model number)


In [9]:
from keras.models import load_model
# from keras.preprocessing import image
import keras.utils as image
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import SciServer.CasJobs as CasJobs # query with CasJobs, the primary database for the SDSS
import SciServer.SkyServer as SkyServer # show individual objects through SkyServer
import SciServer.SciDrive
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from PIL import Image


In [3]:
cnn_model = load_model('pcc_resnet50_4.h5')
cnn_model.load_weights
# cnn_model.summary()


<bound method Model.load_weights of <keras.engine.functional.Functional object at 0x7f0ced914340>>

https://stackoverflow.com/questions/43469281/how-to-predict-input-image-using-trained-model-in-keras

In [3]:
# testDir = 'clean-flags-big-sweep'
# testImgs = glob.glob(os.path.join(testDir, '*.png'))
# img_width, img_height = 200, 200

# predictions = []

# for ind, im in enumerate(testImgs):
# #     print(ind, im)
# #     imPaths.append(im)
#     img = image.load_img(im, target_size=(img_width, img_height))
#     x = image.img_to_array(img)
#     ex = np.expand_dims(x, axis = 0)
#     images = np.vstack([ex])
    
#     pred = cnn_model.predict(ex)
#     pred = np.argmax(pred, axis = 1)
#     predictions.append(pred[0])
    
# #     if ind > 4:
# #         break

In [4]:
# mkdir bigSweepPredictions

In [5]:
# predictDir = 'bigSweepPredictions'

# for ind, im in enumerate(testImgs):
    
#     img = image.load_img(im, target_size=(img_width, img_height))
    
#     plt.figure()
#     plt.imshow(img)
#     plt.title(f'Predicted as {predictions[ind]}')
    
#     basename = os.path.basename(im)
#     filename = os.path.join(predictDir, f'{str(basename[:-4])}_label={str(predictions[ind])}')
# #     print(filename)
#     plt.savefig(f'{filename}.png')
#     plt.close()

In [16]:
# join on specObj to see if missmatch b/t PhotoObj type and SpecObj type
# also want to see if objects in PCC appear

radialSearchSpec = f'SELECT TOP 300 p.objID, p.ra, p.dec, \
 p.modelMag_r, \
 p.modelMag_u - p.modelMag_g as u_g, \
 p.modelMag_g - p.modelMag_z as g_z, \
 p.modelMag_g - p.modelMag_r as g_r, \
 p.modelMag_g - p.modelMag_i as g_i, \
 p.modelMag_r - p.modelMag_i as r_i, \
 p.modelMag_r - p.modelMag_z as r_z, \
 p.petroRad_r, p.flags, dbo.fPhotoFlagsN(p.flags) as flag_text, \
 s.specObjID, s.z, s.zErr, s.zWarning, s.class, s.subClass, \
 N.distance \
FROM\
 photoObj as p \
JOIN SpecObjAll s ON p.objID = s.bestObjID \
JOIN dbo.fGetNearbyObjEq(49.9467, 41.5131, 45) as N ON N.objID = p.objID \
WHERE \
 p.modelMag_r > -14.3*(p.modelMag_g - p.modelMag_z) + 37 \
 and p.modelMag_r < -14.3*(p.modelMag_g - p.modelMag_z) + 46 \
 and p.type = 3 \
ORDER BY distance'

radDf = CasJobs.executeQuery(radialSearchSpec, "dr16")
# radDf

Cluster member redshift range 0.01 < spec-z < 0.033 \
Stars spec-z < 0.01 \
Background spec-z > 0.033

In [22]:
def labeler(num):
    if num > 0.033:
        return(0) # background
        
    elif num > 0.01 and num < 0.033:
        return(1) # members
        
    else:
        return(2) # stars

In [14]:
SkyServer_DataRelease = 'DR16'
img_width, img_height = 200, 200
img_array = SkyServer.getJpegImgCutout(ra=197.614455642896, dec=18.438168853724, width=200, height=200, scale=0.1, 
                                 dataRelease=SkyServer_DataRelease)

outPicTemplate = f'{}'
img0 = Image.fromarray(img_array, 'RGB')
img0.save(f'{outDir}/.png')



1/1 [==============================] - 1s 601ms/step


In [ ]:
### Populating Test set ###
outDir = 'Radial-Search-2_2024'
for id, z, r, d in zip(radDf['objID'], radDf['z'], radDf['ra'], radDf['dec']):
    img_array = SkyServer.getJpegImgCutout(ra=r, dec=d, width=200, height=200, scale=0.1, 
                                 dataRelease=SkyServer_DataRelease)
    # print(f'{id}-label={labeler(z)}')
    outPicTemplate = f'{id}-label={labeler(z)}.png'
    
    img0 = Image.fromarray(img_array, 'RGB')
    img0.save(f'{outDir}/.png')

In [15]:
# img0

# img = image.load_img(img0, target_size=(img_width, img_height))
x = image.img_to_array(img0)
ex = np.expand_dims(x, axis = 0)
images = np.vstack([ex])
pred = cnn_model.predict(ex)
pred = np.argmax(pred, axis = 1)
pred

array([0])